# From Datetime to MEM format

Take normal datetime format of the pandas equivalent and covert
the file to MEM style: year, month, day, hour format


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import csv
import matplotlib.pyplot as plt

In [ ]:
def make_MEM_compatible(f_name, save_name, date_time_field='date_time'):
    print(f"In file: {f_name}")
    df = pd.read_csv(f_name)
    df['date_time'] = pd.to_datetime(df[date_time_field])
    
    with open(f'{save_name}.csv', 'w', newline='') as csvfile:

        fieldnames = ['year', 'month', 'day', 'hour', 'demand (MW)']#'date_time']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for idx in df.index:
            # This strange shift is to uses MEM's 1-24 counting,
            # see example datetime and MEM for these three hours:
            # year,month,day,hour,date_time
            # 2015,7,2,23,2015-07-02 23:00:00
            # 2015,7,2,24,2015-07-03 00:00:00
            # 2015,7,3,1,2015-07-03 01:00:00
            mem_format = df.loc[idx, 'date_time'] + timedelta(hours=-1)
            writer.writerow({
                'year': mem_format.year,
                'month': mem_format.month,
                'day': mem_format.day,
                'hour': mem_format.hour+1,
                'demand (MW)': int(df.loc[idx, 'cleaned demand (MW)']),
                #'date_time': df.loc[idx, 'date_time']
            })
    print(f"Outfile: {save_name}")

In [ ]:

base = '~/Downloads/truggles-EIA_Cleaned_Hourly_Electricity_Demand_Data-1b31ad5/data/release_2019_Oct/'

make_MEM_compatible(base+'contiguous_US/CONUS.csv', 'US_demand_unnormalized')

make_MEM_compatible(base+'interconnects/EASTERN_IC.csv', 'EASTERN_IC')
make_MEM_compatible(base+'interconnects/TEXAS_IC.csv', 'TEXAS_IC')
make_MEM_compatible(base+'interconnects/WESTERN_IC.csv', 'WESTERN_IC')

In [ ]:
def compare_with_previous_file(old, o_header, new, n_header):
    
    o = pd.read_csv(old, header=o_header)
    n = pd.read_csv(new, header=n_header)
    
    print(len(n.index))
    
    n = n.iloc[1:-1]
    n = n.reset_index()
    
    o = o.iloc[0:35062]
    o = o.reset_index()
    
    print(o.head())
    print(n.head())
    
    print(o.tail())
    print(n.tail())
    
    fig, axs = plt.subplots(1,2)
    axs[0].hist(o['demand (MW)']/n['demand (MW)'], 50)
    
    axs[1].hist(o['demand (MW)']/n['demand (MW)'], 50)
    plt.yscale('log')
    
    axs[0].set_ylabel('entries (4 years total)')
    axs[0].set_xlabel('previous value/new value')
    axs[1].set_ylabel('entries (4 years total)')
    axs[1].set_xlabel('previous value/new value')
    
    plt.tight_layout()
    plt.show()
    
compare_with_previous_file('/Users/truggles/SEM-1.2_CIW/Input_Data/Lei_Solar_Wind/US_demand_unnormalized.csv',
                          8,
                          'US_demand_unnormalized.csv', 0)